# Lire des données

**Références** : https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html

Chaque type de données est différent et nécessitera une méthodede chargement adapté. De plus chaque format de donnée admets plusieurs variantes qu'il faudra également contrôler.

> **règle d'or** : TOUJOURS vérifier les données que l'on a chargé !

Il peut se passer tout un tas de problèmes comme (liste non exhaustive) :
* un nombre chargé comme une chaine de caractère
* un csv dont le séparateur est le `';'` plutôt que la `','`(un classique d'excel),
* un csv dont la première ligne et (*resp.* n'est pas) le nom des colonnes
* l'encodage du texte qui n'est pas l'[utf-8](https://fr.wikipedia.org/wiki/UTF-8) un autre classique de d'excel
* le séparateur de décimal qui dans le monde est le `'.'` et en France la `','`
* un séparateur de millier (un `' '`, parfois la `','`
* ...

Bref ça peut merder partout. Et faire un jour ou une semaine d'analyse pour se rendre compte que l'on a fait tout ça sur queque chose qui n'étaitpas les données, c'est rageant. Et c'est en core plus rageant quand quelqu'un d'autre s'en rend compte en essayant de vérifier vos résultats.

## datasets

Pour s'entrainer ou tester des choses sur des jeu de donnée jouet, on peut utiliser ceux mis à disposition par les bibliothèques.


### dans des bibliothèques pythons

Déjà packagées dans des bibliothèques, comme *seaborn* ou *scikit-lean*

#### seaborn

https://github.com/mwaskom/seaborn-data

In [ ]:
import seaborn

les différents jeux de données :

In [ ]:
seaborn.get_dataset_names()

les iris, comme on l'a déjà vu :

In [ ]:
iris = seaborn.load_dataset('iris')
iris

#### scikit-learn

https://scikit-learn.org/stable/datasets/index.html

Attention, le format n'est pas du tout au format pandas. Il va falloir les importer.

In [ ]:
import sklearn.datasets

In [ ]:
wine = sklearn.datasets.load_wine()

Les données sklearn sont dans leur format à eux. C'est un dictionnaire qui contient les données :

In [ ]:
print(wine.keys())

les clés `DESCR`et `feature_names` décrivent les données :

In [ ]:
print(wine['DESCR'])

In [ ]:
print(wine['feature_names'])

Les données en eux-mêmes sont dans la clé `data` et sont stockées sous la forme d'une matrice toute bête.

In [ ]:
print(wine['data'])

Pour bénéficier des bibliothèques de traitement de données de pandas, il faut fabriquer un dataframe (en gros une matrice dont les lignes et colonnes sont nommées).

In [ ]:
import pandas

wine2 = pandas.DataFrame(wine['data'], columns=wine['feature_names'])

wine2

### Sites

De nombreux site proposent des données

#### UCI

https://archive.ics.uci.edu/ml/index.php

Par exemple https://archive.ics.uci.edu/ml/datasets/Car+Evaluation dont les données osnt là : https://archive.ics.uci.edu/ml/machine-learning-databases/car/

> **Attention** ce site contient des données depuis longtmeps. Leur format peut donc être vraiment exotique. Il faut faire très très attention

In [ ]:
import urllib.request

print(urllib.request
        .urlopen('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.names')
        .read()
        .decode('ascii'))

Transformer des données d'un format à un autre peut être piégeux. Il faut être extrêment méfiant et scupuleux. Dans l'exemple ci-après :

- attention aux nombre de colonnes (il y en a une de plus qu'annoncé). Il y a une colonne de prédiction en plus.
- il y a plein de variante de csv. Ici il n'y a pas de délimiteur de chaine de caractères (usuellement ")
- on a l'impression que les colonnes 3 et 4 sont des entiers, ce n'est pas le cas (regardez les dernières lignes).
- attention au nom des colonnes. Sans "header=None", la première ligne devient le nom des colonnes

In [ ]:
cars = pandas.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data',
                       header=None,
                       names=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'value'])

print(cars)

#### Autres sites

* https://www.data.gouv.fr/fr/
* https://catalog.data.gov/dataset
* http://opendata.maregionsud.fr/
* https://opendata.paris.fr/pages/home/
* https://www.insee.fr/fr/statistiques
* ...


## lire et exporter des données excel

Pandas peut : 
- lire des fichiers excels : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
- ecrire des fichiers excel : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html

Pour modifier des fichiers excel on préfèrera utiliser des bibliothèques spécialisées comme https://openpyxl.readthedocs.io/en/stable/

## csv

Pour Comma Separated Value : https://fr.wikipedia.org/wiki/Comma-separated_values

Mais **attention** : il y a plein de variantes!

* excel en français va mettre des ";" à la place des ","
* parfois c'est des tabulations comme séparateur, on parle alors de TSV
* 1ère ligne : est-ce des données ou le nom des champs ?
* ...


**Bref** : comme à chaque fois que vous voulez importer des données, il faut fire très très attention. Dans l'exemple ci-dessous :
* le séparateur est un ";"
* l'encoding du texte est spécial.


In [ ]:
import pandas

In [ ]:
# https://www.data.gouv.fr/fr/datasets/liste-de-prenoms/

csv_prenom = pandas.read_csv("https://www.data.gouv.fr/fr/datasets/r/55cd803a-998d-4a5c-9741-4cd0ee0a7699",
                             sep=";",
                             encoding="ISO-8859-1")
csv_prenom

## json url

Le format https://www.json.org est un format populaire de transmission de données. C'est **une chaine de caractère** qui est transformée en données (de type entier, réel, etc).

On peut voir ça comme un dictionnaire  ou une liste façon javascript (https://la-cascade.io/json-pour-les-debutants/). Python utilise nativement le json via sa bibliothèque json (https://docs.python.org/3/library/json.html).

pandas permet d'importer des données décrite sous la forme d'un objet json où chaque entrée est une colonne.

### json, dictionnaires et importation

**Attention** à pas confondre les données écrites sous forme json **qui sont des chaines de caractères** et leurs interprétations dans le langage pyhton **sous la forme de dictionnaire**.

#### Des données sous la forme d'un dictionnaire python

In [ ]:
mes_donnees_dictionnaire = {
    "x": [1, 2, 3, 4, 5],
    "y": [-1, -2, -3, -4, -5]
}

#### Les même donées au format json (une chaine de caractère)

In [ ]:
import json

json.dumps(mes_donnees_dictionnaire)

#### Importation d'un json en pandas

In [ ]:
json_data = pandas.read_json(json.dumps(mes_donnees_dictionnaire))

print(json_data)

La ligne suivante (si on la décommente) ne fonctionne en revanche pas :

In [ ]:
# pandas.read_json(mes_donnees_dictionnaire)

On peut aussi charger un json directement depuis une url :  https://chrisalbon.com/python/data_wrangling/load_json_file_into_pandas/